In [4]:
import pandas as pd
import json
from datetime import datetime
import uuid

print("--- INICIANDO REVERSE ETL (Simulación SFMC) ---")

# 1. Cargar la lista de acciones (Output del Paso 3)
try:
    df_actions = pd.read_csv('Final_Action_List.csv')
    print(f"Input cargado: {len(df_actions)} registros.")
except FileNotFoundError:
    print("Error: Falta 'Final_Action_List.csv'. Ejecuta el Paso 3.")
    exit()

# ---------------------------------------------------------
# A. GENERAR 'DE_TARGET_AUDIENCE.csv' (Formato SFMC)
# ---------------------------------------------------------
# Salesforce requiere claves exactas. Vamos a mapear nuestras acciones
# a códigos técnicos y líneas de asunto dinámicas.

# Mapeo de Subject Lines (Creatividad dinámica)
subject_lines = {
    "SMS_RECOVER_CART_20_OFF": "SL_URGENCY_001",   # "¡Vuelve! Tu cesta expira en 2h + 20% OFF"
    "PUSH_NEW_COLLECTION_NO_PROMO": "SL_VIP_005",  # "Exclusivo para ti: Nueva Colección Otoño"
    "EMAIL_WE_MISS_YOU_STORYTELLING": "SL_EMO_009",# "¿Recuerdas tu última visita, [Name]?"
    "EMAIL_WEEKLY_NEWSLETTER": "SL_GEN_001",       # "Las tendencias de la semana ya están aquí"
    "DO_NOTHING_NEGATIVE_VALUE": "N/A",
    "DO_NOTHING_LOW_SCORE": "N/A"
}

# Filtramos a los que NO vamos a contactar (DO_NOTHING)
# En Marketing Cloud solo subimos a quien queremos impactar hoy.
df_upload = df_actions[~df_actions['Next_Best_Action'].str.startswith("DO_NOTHING")].copy()

# Mapeamos la columna Subject Line ID
df_upload['Dynamic_Subject_Line_ID'] = df_upload['Next_Best_Action'].map(subject_lines)

# Renombramos columnas al estándar de Salesforce (PascalCase suele ser norma)
# user_id -> SubscriberKey (La clave primaria sagrada de SFMC)
# Next_Best_Action -> NBA_Code
df_final = df_upload.rename(columns={
    'user_id': 'SubscriberKey',
    'email_clean': 'EmailAddress',
    'Next_Best_Action': 'NBA_Code'
})

# Selección final de columnas (Orden estricto)
cols_sfmc = ['SubscriberKey', 'EmailAddress', 'NBA_Code', 'Dynamic_Subject_Line_ID']
df_final = df_final[cols_sfmc]

# Guardar CSV final
filename_csv = 'DE_Target_Audience.csv'
df_final.to_csv(filename_csv, index=False)
print(f"\n[OK] Archivo '{filename_csv}' generado con {len(df_final)} filas.")
print("    -> Listo para ingesta en Automation Studio (SFTP).")

# ---------------------------------------------------------
# B. GENERAR 'TRIGGER_LOG.json' (Auditoría Técnica)
# ---------------------------------------------------------
# Esto es lo que revisa el equipo de IT si algo falla.
# Demuestra que controlas el proceso, no solo el dato.

audit_log = {
    "job_id": str(uuid.uuid4()),
    "timestamp_utc": datetime.utcnow().isoformat() + "Z",
    "process_name": "ETL_Daily_NBA_Scoring",
    "source_system": "DuckDB_Local_Analytics",
    "target_system": "SFMC_Data_Extension_Audience",
    "metrics": {
        "total_input_rows": len(df_actions),
        "total_output_rows": len(df_final), # Enviados
        "filtered_rows": len(df_actions) - len(df_final), # Ignorados (Do Nothing)
        "execution_status": "SUCCESS"
    },
    "schema_validation": {
        "columns_checked": cols_sfmc,
        "null_check": "PASSED"
    }
}

filename_json = 'Trigger_Log.json'
with open(filename_json, 'w') as f:
    json.dump(audit_log, f, indent=4)

print(f"\n[OK] Log de auditoría '{filename_json}' generado.")
print("    -> Simula respuesta de API o log de Airflow.")

# Ver contenido del JSON
print("\n--- CONTENIDO DEL LOG ---")
print(json.dumps(audit_log, indent=4))


--- INICIANDO REVERSE ETL (Simulación SFMC) ---
Input cargado: 510 registros.

[OK] Archivo 'DE_Target_Audience.csv' generado con 299 filas.
    -> Listo para ingesta en Automation Studio (SFTP).

[OK] Log de auditoría 'Trigger_Log.json' generado.
    -> Simula respuesta de API o log de Airflow.

--- CONTENIDO DEL LOG ---
{
    "job_id": "d4585ec8-5230-4206-9d48-e841fae5791f",
    "timestamp_utc": "2026-01-07T14:27:11.717989Z",
    "process_name": "ETL_Daily_NBA_Scoring",
    "source_system": "DuckDB_Local_Analytics",
    "target_system": "SFMC_Data_Extension_Audience",
    "metrics": {
        "total_input_rows": 510,
        "total_output_rows": 299,
        "filtered_rows": 211,
        "execution_status": "SUCCESS"
    },
    "schema_validation": {
        "columns_checked": [
            "SubscriberKey",
            "EmailAddress",
            "NBA_Code",
            "Dynamic_Subject_Line_ID"
        ],
        "null_check": "PASSED"
    }
}


C:\Users\bruno\AppData\Local\Temp\ipykernel_560\1175574589.py:66: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_utc": datetime.utcnow().isoformat() + "Z",
